In [6]:
import cv2
import os
from PIL import Image
import torch
import numpy as np

def CatchUsbVideo(window_name, camera_idx):
  face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

  cap = cv2.VideoCapture(camera_idx)

  while cap.isOpened():

    # 截取保存这一帧
    for i in range(500):
      # 持续读取新帧
      ret, frame = cap.read()
      # 显示新帧 
      cv2.imshow(window_name, frame)
      cv2.waitKey(5)
      if (i+1)%10 == 0:
              img_name = "test_{}.jpg".format(int((i-9)/10))
              cv2.imwrite(os.path.join(dataset_path, img_name), frame)
   
    break
  

dataset_path = './test'

camera = cv2.VideoCapture(0,cv2.CAP_DSHOW)
ret, frame = camera.read()

name = input('请输入姓名:')

# 展示摄像头画面  
CatchUsbVideo("camera", 0)
# 2秒后关闭窗口   
camera.release()
cv2.destroyAllWindows() 

In [7]:
from facenet_pytorch import MTCNN, InceptionResnetV1

# If required, create a face detection pipeline using MTCNN:
#mtcnn = MTCNN(image_size=224*224, margin=10)

# Create an inception resnet (in eval mode):
resnet = InceptionResnetV1(pretrained='vggface2').eval()

In [8]:
import cv2
import numpy as np
train_encode=[]
for i in range(20):
      img = cv2.imread("./face_dataset/{}_{}.jpg".format(name,int(i)))
      face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
      gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      faces = face_cascade.detectMultiScale(gray)
      if len(faces)==1:
           (x, y, w, h)  = faces[0]
           w += int(w*0.1)
           h += int(h*0.1)
           face = img[y:y+h, x:x+w]
           train_encode.append(face)
      else:
            continue
 
# 检查每个元素的形状
shapes = [item.shape for item in train_encode]
# 确保所有元素具有相同的形状
if len(set(shapes)) > 1:
    # 调整元素的形状为相同的形状
    max_shape = np.max(shapes, axis=0)
    train_encode = [np.resize(item, max_shape) for item in train_encode]

# 将图像数据调整为符合模型期望的形状和类型
img = np.mean(train_encode, axis=0)
img_tensor = torch.from_numpy(img)
# 预处理数据
img_tensor = img_tensor.float() # 转换为float
img_tensor = img_tensor.permute(2, 0, 1) # 调整通道顺序
img_tensor = img_tensor.unsqueeze(0) # 增加batch维度
# Calculate embedding (unsqueeze to add batch dimension)
train_embedding = resnet(img_tensor)
train_embedding = train_embedding.detach().numpy()
print(train_embedding)

[[ 5.31880148e-02 -4.36835289e-02  1.92507971e-02  4.85289246e-02
   5.03070913e-02 -3.85250971e-02 -2.58633457e-02 -2.45009491e-04
  -3.45013849e-02 -2.27505025e-02 -6.73175007e-02  3.19619626e-02
  -7.65109956e-02 -3.66707356e-03  5.41113429e-02 -5.84983416e-02
   6.63091475e-03  1.52598899e-02  3.47180143e-02  6.01059897e-03
   1.27102686e-02  4.15045433e-02 -2.06512846e-02 -1.01254843e-02
  -3.04158730e-03  1.88395455e-02  1.05674535e-01 -4.02253075e-03
   3.56194600e-02 -8.76044184e-02 -1.73922349e-03  1.34179872e-02
   4.57489565e-02  1.39948903e-02  4.16896753e-02 -2.68221507e-03
  -1.07152402e-01  3.82289104e-02 -5.90815954e-02  5.97979054e-02
  -2.96630412e-02  1.43813230e-02  1.85214318e-02 -1.14179023e-01
  -2.51362994e-02 -3.67749371e-02  2.40907483e-02  1.80015955e-02
  -8.75281841e-02  3.28683928e-02  8.98839161e-02  1.02415383e-01
   9.42666605e-02  4.43025045e-02 -3.12035978e-02 -5.63169718e-02
   4.92422096e-02  1.00361049e-01 -3.23931910e-02  3.59342992e-02
  -2.84703

In [9]:
#对需要验证的人脸编码
import cv2
import numpy as np
var_encode=[]
for i in range(50):
     img = cv2.imread("./test/test_{}.jpg".format(int(i)))
     face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
     faces = face_cascade.detectMultiScale(gray)
     if len(faces)==1:
           (x, y, w, h)  = faces[0]
           w += int(w*0.1)
           h += int(h*0.1)
           face = img[y:y+h, x:x+w]
           var_encode.append(face)
     else:
            continue

length = len(var_encode)
print(length)
if length<10:
     print ("请勿遮挡面部或侧对摄像头并重新验证")
# 检查每个元素的形状
shapes = [item.shape for item in var_encode]
# 确保所有元素具有相同的形状
if len(set(shapes)) > 1:
    # 调整元素的形状为相同的形状
    max_shape = np.max(shapes, axis=0)
    var_encode = [np.resize(item, max_shape) for item in var_encode]   

var_encoding = []
for _,img in  enumerate(var_encode):
     img_tensor = torch.from_numpy(img)
     # 预处理数据
     img_tensor = img_tensor.float() # 转换为float
     img_tensor = img_tensor.permute(2, 0, 1) # 调整通道顺序
     img_tensor = img_tensor.unsqueeze(0) # 增加batch维度
     # Calculate embedding (unsqueeze to add batch dimension)
     var_embedding = resnet(img_tensor)
     var_embedding = var_embedding.detach().numpy()
     var_encoding.append(var_embedding)

print(var_encoding[0])
print(var_encoding[1])




     
     

46
[[ 6.05531782e-02 -1.72628276e-02  1.89942289e-02  3.27204019e-02
   4.61072028e-02 -4.87621203e-02 -3.42629030e-02  1.52771017e-02
  -2.33918726e-02 -5.50812222e-02 -9.11234543e-02  1.63207192e-03
  -8.65429789e-02 -1.04627069e-02  6.58110753e-02 -7.37738237e-02
  -1.40638826e-02 -1.36520350e-02  2.72306148e-02 -1.56984245e-03
   1.99804846e-02  5.57904243e-02 -3.57053950e-02 -1.27115995e-02
   1.59233734e-02  2.62886807e-02  1.18456885e-01 -1.68454386e-02
   3.59232053e-02 -8.01257417e-02 -2.84411525e-03  3.15253697e-02
   4.79449593e-02  2.81447116e-02  4.10775878e-02  1.63210016e-02
  -8.92195404e-02  5.61142489e-02 -4.46862355e-02  3.57520618e-02
  -1.91657580e-02  1.20970281e-02  8.02118424e-03 -1.18669525e-01
  -3.50575671e-02 -6.39442056e-02  6.16604788e-03 -3.76427313e-03
  -8.39298964e-02  1.53335426e-02  8.84161368e-02  8.74291062e-02
   8.06564987e-02  4.43182327e-02 -2.94063445e-02 -3.89807932e-02
   4.92236875e-02  8.70183930e-02 -1.67009681e-02  1.98172592e-02
  -2.78

In [10]:
similarity = []
count = 0
for i in range(length):
    sim = np.dot(train_embedding, var_encoding[i].T) / (np.linalg.norm(train_embedding) * np.linalg.norm(var_encoding[i]))
    similarity.append(sim)
    if float(sim) > 0.75:
        count += 1
print(similarity)
if count/length > 0.6:
    print("启动！")
else:
    print("未能识别，请重试")



[array([[0.94395095]], dtype=float32), array([[0.9222743]], dtype=float32), array([[0.91071343]], dtype=float32), array([[0.92558026]], dtype=float32), array([[0.91630834]], dtype=float32), array([[0.91405034]], dtype=float32), array([[0.9182148]], dtype=float32), array([[0.9230008]], dtype=float32), array([[0.9325877]], dtype=float32), array([[0.91945136]], dtype=float32), array([[0.9176176]], dtype=float32), array([[0.91366917]], dtype=float32), array([[0.9280514]], dtype=float32), array([[0.9298101]], dtype=float32), array([[0.8744646]], dtype=float32), array([[0.9141673]], dtype=float32), array([[0.9164614]], dtype=float32), array([[0.88329893]], dtype=float32), array([[0.93744975]], dtype=float32), array([[0.9146601]], dtype=float32), array([[0.9244984]], dtype=float32), array([[0.8922496]], dtype=float32), array([[0.8940086]], dtype=float32), array([[0.93618166]], dtype=float32), array([[0.9206274]], dtype=float32), array([[0.91537786]], dtype=float32), array([[0.92169404]], dtyp